In [1]:
import numpy as np
import os
import masknmf
import sys
import torch

import tifffile
import fastplotlib as fpl
import matplotlib.pyplot as plt

import time
from typing import *
%matplotlib inline
%load_ext autoreload

fastplotlib version from git (0.2.0) and __version__ (0.5.0) don't match.
No windowing system present. Using surfaceless platform
No config found!
No config found!
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Valid,Device,Type,Backend,Driver
✅ (default),NVIDIA TITAN RTX,DiscreteGPU,Vulkan,555.42.02
❗ limited,"llvmpipe (LLVM 12.0.0, 256 bits)",CPU,Vulkan,Mesa 21.2.6 (LLVM 12.0.0)
❌,NVIDIA TITAN RTX/PCIe/SSE2,Unknown,OpenGL,3.3.0 NVIDIA 555.42.02


Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


In [2]:
# Load the masknmf moco 
masknmfmoco_file = "your_file"
data = tifffile.imread(masknmfmoco_file)

In [3]:
full_moco_dense = data

In [4]:
pmd_arr_moco = masknmf.compression.pmd_decomposition(full_moco_dense,
                                                [20, 20],
                                                full_moco_dense.shape[0],
                                                max_components = 20,
                                                max_consecutive_failures=3,
                                                background_rank = 4,
                                               device = "cuda",
                                               pixel_weighting=None,
                                               frame_batch_size = 1024)

[25-05-11 06:35:18]: Starting compression
[25-05-11 06:35:18]: sampled from the following regions: [0]
[25-05-11 06:35:18]: We are initializing on a total of 15957 frames
[25-05-11 06:35:19]: Approximating full FOV basis terms
[25-05-11 06:35:20]: Loading data to estimate complete spatial basis
[25-05-11 06:35:24]: Finding spatiotemporal roughness thresholds


100%|████████████████████████████████████████| 250/250 [00:00<00:00, 252.81it/s]

[25-05-11 06:35:25]: Running Blockwise Decompositions


[25-05-11 06:35:46]: Constructed U matrix. Rank of U is 10133
[25-05-11 06:35:48]: PMD Objected constructed


In [5]:
pmd_arr_moco.shape

(15957, 328, 524)

In [6]:
pmd_arr_moco.to('cuda')
pmd_arr_moco.rescale = False

In [7]:
full_moco_dense.shape

(15957, 328, 524)

In [8]:
from functools import partial
from ipywidgets import VBox, HBox
from collections import OrderedDict

In [30]:
iw = fpl.ImageWidget(
    [full_moco_dense, pmd_arr_moco],
    names=["mcorr", "pmd"],
)

mcorr_img = iw.managed_graphics[0]
pmd_img = iw.managed_graphics[1]

image_graphics = [
    mcorr_img,
    pmd_img,
]

fig_temporal = fpl.Figure(shape=(2, 1), names=["mcorr", "pmd"])
fig_temporal["mcorr"].add_line(np.zeros(full_moco_dense.shape[0]))
fig_temporal["pmd"].add_line(np.zeros(pmd_arr_moco.shape[0]))

mcorr_selectors = list()
pmd_selectors = list()

rect_selector_kwargs = dict(
    edge_thickness=1, 
    edge_color="w", 
    vertex_thickness=3.0, 
    vertex_color="cyan"
)

selectors = OrderedDict()

for img in image_graphics:
    selectors[img] = list()

# mcorr_selectors.append(
#     mcorr_img.add_rectangle_selector(
#         selection=[0, 20, 0, 20],
#         **rect_selector_kwargs
#     )
# )
# pmd_selectors.append(
#     pmd_img.add_rectangle_selector(
#         selection=[0, 20, 0, 20],
#         **rect_selector_kwargs
#     )
# )

# selectors = [
#     mcorr_selectors,
#     pmd_selectors,
# ]

def rect_selector_moved(selectors_pair: tuple[fpl.RectangleSelector], ev: fpl.GraphicFeatureEvent):
    for selector in selectors_pair:
        selector.selection = ev.info["value"]
    
    row_ixs, col_ixs = ev.get_selected_indices()
    row_slice = slice(row_ixs[0], row_ixs[-1] + 1)
    col_slice = slice(col_ixs[0], col_ixs[-1] + 1)

    mcorr_temporal = full_moco_dense[:, row_slice, col_slice].mean(axis=(1, 2)).ravel()
    pmd_temporal = pmd_arr_moco[:, row_slice, col_slice].mean(axis=(1, 2)).ravel()

    fig_temporal["mcorr"].graphics[0].data[:, 1] = mcorr_temporal
    fig_temporal["pmd"].graphics[0].data[:, 1] = pmd_temporal

    for subplot in fig_temporal:
        subplot.auto_scale()

# for pair in zip(*selectors):
#     for sel in pair:
#         sel.add_event_handler(partial(rect_selector_moved, pair), "selection")

HBox([iw.show(), fig_temporal.show(maintain_aspect=False)])

RFBOutputContext()

RFBOutputContext()

In [31]:
import pygfx

In [32]:
RESIZING_NEW_RECT = False


def add_rectangle(ev: pygfx.PointerEvent):
    global RESIZING_NEW_RECT
    
    if ev.button != 1:
        return

    for subplot in iw.figure:
        subplot.controller.enabled = False
        
    # in world space
    x, y = ev.pick_info["index"]

    new_selectors = list()
    
    for img in image_graphics:
        new_selector = img.add_rectangle_selector(
            selection=[x, x + 1, y, y + 1],
            **rect_selector_kwargs
        )

        selectors[img].append(new_selector)
        new_selectors.append(new_selector)

    for sel in new_selectors:
        sel.add_event_handler(partial(rect_selector_moved, new_selectors), "selection")

    RESIZING_NEW_RECT = True


for img in image_graphics:
    img.add_event_handler(add_rectangle, "pointer_down")


@iw.figure.renderer.add_event_handler("pointer_move")
def resize_rect(ev: pygfx.PointerEvent):
    global RESIZING_NEW_RECT
    
    if not RESIZING_NEW_RECT:
        return

    img = image_graphics[0]

    for subplot in iw.figure:
        # world (x, y)
        pos = subplot.map_screen_to_world(ev)
        if pos is None:
            continue
        else:
            break

    if pos is None:
        # if pointer was moved outside the subplot
        RESIZING_NEW_RECT = False
        return

    x2, y2, _ = pos

    # most recently added selector
    x1, _, y1, _ = selectors[img][-1].selection
    
    selectors[img][-1].selection = [x1, x2, y1, y2]


@iw.figure.renderer.add_event_handler("pointer_up")
def end_resize(ev: pygfx.PointerEvent):
    global RESIZING_NEW_RECT
    
    if ev.button != 1:
        return
    if not RESIZING_NEW_RECT:
        return

    for subplot in iw.figure:
        subplot.controller.enabled = True
    
    RESIZING_NEW_RECT = False